In [ ]:
import numpy as np
import math
import plotly.graph_objects as go
#Angle directions uses conventions on that page  --> https://en.wikipedia.org/wiki/Spherical_coordinate_system
class HalbachArray:  #Our array is placed on x-y plane,and the circular gap of array is in z axis.
    full_angle=2*math.pi  #Angle of full circle
    B_strength=1.45 #Tesla// I took values from here --> https://www.stanfordmagnets.com/what-is-the-difference-between-n35-and-n52-magnets.html
    #Dimesions of magnet 10*10*10 mm
    def __init__(self,n_of_magnets,radius_of_array,dimensions_of_square_screen,resolution,theta_array,phi_array): #Resolution is how much data points we have measured inside of the viewing screen of magnetic field.
        self.n_of_magnets=n_of_magnets #n_of_magnets is the number of magnets in halbach array 
        self.radius_of_array=radius_of_array
        self.dimensions_of_square_screen=dimensions_of_square_screen #This is the place where we put magnetic bots (For our purpose we will see it as a viewing screen of magnetic field)
        self.resolution=resolution
        self.theta_array=np.deg2rad(np.array(theta_array)) #These are the orientation angles for each magnet we used in hallbach array.From zero'th magnet to nth magnet,we are entering each angle here.
        self.phi_array=np.deg2rad(np.array(phi_array)) #Angles will taken as degrees and converted here to radians for proper calculation
        
    def is_ok(self): #Check whether input parameters are correct
        if len(self.theta_array)==self.n_of_magnets and len(self.phi_array)==self.n_of_magnets:
            return True
        else:
            return False
    def coordinates_of_magnets(self): #Each magnet is called with its number.First magnet is numbered as zero.Additionally,first magnet is placed at angle zero in polar coordinates and remaining ones are placed in counter-clockwise direction properly.
        
        coordinates=[]
        angle_increment=(self.full_angle)/self.n_of_magnets
        for i in range(self.n_of_magnets):
            coordinates.append([self.radius_of_array*math.cos(0+i*angle_increment),self.radius_of_array*math.sin(0+i*angle_increment)])
        return np.array(coordinates) #The function returns coordinate arrays at the end
    def plot_magnetic_field_screen(self): #It will plot the magnetic field screen with seismic colormap #It will calculate and plot magnetic field via plotly 3D cone plot.
        #We start by defining r unit vector to assign a direction to our magnetic dipole and r_x denotes x component of r.
        m_x=[]
        m_y=[]
        m_z=[]
        for i in range(len(self.theta_array)):
            m_x.append(1*np.sin(self.theta_array[i])*np.cos(self.phi_array[i])) # r=1
            m_y.append(1*np.sin(self.theta_array[i])*np.sin(self.phi_array[i]))
            m_z.append(1*np.cos(self.theta_array[i]))
        #Now define m vector which is valid for each magnet,each 

        #Now we have calculated magnitudes of our direction vector r for our magnetic dipole,we are ready to dive in our magnetic field calculation and plotting
        m_x=np.array(m_x)
        m_y=np.array(m_y)
        m_z=np.array(m_z)

        #Now calculate positions of r vectors

        
        
        
        

        




        


## To calculate magnetic field of each dipole from distance r,we use 
$\mathbf{B}(\mathbf{r}) = \nabla \times \mathbf{A} = \frac{\mu_0}{4\pi} \left[ \frac{3\mathbf{r}(\mathbf{m} \cdot \mathbf{r})}{r^5} - \frac{\mathbf{m}}{r^3} \right]$
